In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
root = '/content/drive/MyDrive/IGP_Assignment/csv/dataset.xlsx'
data = pd.read_excel(root)
data.head()

,text,target
0,Customized Romantic Quote Wooden Frame,"desk accessories,stationery and desk accessories"
1,send birthday cake and flower online,"cakes,photo cakes,designer cakes,regular cakes..."
2,Someone I Love Birthday Cake (1 kg),"cakes,photo cakes"
3,pig,none
4,bags,"bags and clutches,laptop bags,handbags,sling b..."


In [ ]:
data.shape

(44886, 2)

In [ ]:
def remove_none_row(data):
  for idx, ele in enumerate(data['target']):
    if ele == 'none':
      data = data.drop(idx)
  return data

def count_none_row(data):
  count_none = 0
  for i in data['target']:
    if i == 'none':
      count_none = count_none + 1
  return count_none

def data_cleaning(PATH):
  data = pd.read_excel(PATH)
  print("data shape", data.shape)
  print("Number of NONE value in data", count_none_row(data))

  print("----------------After Cleaning data ----------------")

  data = remove_none_row(data)

  print("data shape", data.shape)
  print("Number of NONE value in data", count_none_row(data))

  return data


In [ ]:
data = data_cleaning(root)

data shape (44886, 2)
Number of NONE value in data 12923
----------------After Cleaning data ----------------
data shape (31963, 2)
Number of NONE value in data 0


In [ ]:
data.head()

,text,target
0,Customized Romantic Quote Wooden Frame,"desk accessories,stationery and desk accessories"
1,send birthday cake and flower online,"cakes,photo cakes,designer cakes,regular cakes..."
2,Someone I Love Birthday Cake (1 kg),"cakes,photo cakes"
4,bags,"bags and clutches,laptop bags,handbags,sling b..."
6,perfume for dad,"beauty and personal care,deos and perfumes"


In [ ]:
def text_cleaning(data):
  li = []
  for i in data.index:
    li.append(data['target'][i].replace(' and ',',')) 
  data['clean_target'] = li
  li1 = []
  for i in data.index:
    li1.append(data['clean_target'][i].replace(' & ',',')) 
  data['clean_target'] = li1

  return data

In [ ]:
data = text_cleaning(data)

In [ ]:
data.head()

,text,target,clean_target
0,Customized Romantic Quote Wooden Frame,"desk accessories,stationery and desk accessories","desk accessories,stationery,desk accessories"
1,send birthday cake and flower online,"cakes,photo cakes,designer cakes,regular cakes...","cakes,photo cakes,designer cakes,regular cakes..."
2,Someone I Love Birthday Cake (1 kg),"cakes,photo cakes","cakes,photo cakes"
4,bags,"bags and clutches,laptop bags,handbags,sling b...","bags,clutches,laptop bags,handbags,sling bags,..."
6,perfume for dad,"beauty and personal care,deos and perfumes","beauty,personal care,deos,perfumes"


In [ ]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import make_scorer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import hamming_loss
from sklearn import metrics
from sklearn.metrics import f1_score, precision_score, recall_score

In [ ]:
def clean_text(text):
    text = text.lower()
    text = text.strip(' ')
    return text

In [ ]:
data['text'] = data['text'].apply(lambda x: clean_text(x))

In [ ]:
data['text']

0                   customized romantic quote wooden frame
1                     send birthday cake and flower online
2                      someone i love birthday cake (1 kg)
4                                                     bags
6                                          perfume for dad
                               ...                        
44880                   dream big personalized photo frame
44881                           personalised led key chain
44882                    online flower delivery in gurgaon
44883    celestial vanilla almond cake (eggless) (half kg)
44885                    set of 3 personalized photo frame
Name: text, Length: 31963, dtype: object

In [ ]:
import ast
data['clean_target'] = data['clean_target'].apply(lambda x : (x.split(",")))

In [ ]:
data.head()

,text,target,clean_target
0,customized romantic quote wooden frame,"desk accessories,stationery and desk accessories","[desk accessories, stationery, desk accessories]"
1,send birthday cake and flower online,"cakes,photo cakes,designer cakes,regular cakes...","[cakes, photo cakes, designer cakes, regular c..."
2,someone i love birthday cake (1 kg),"cakes,photo cakes","[cakes, photo cakes]"
4,bags,"bags and clutches,laptop bags,handbags,sling b...","[bags, clutches, laptop bags, handbags, sling ..."
6,perfume for dad,"beauty and personal care,deos and perfumes","[beauty, personal care, deos, perfumes]"


In [ ]:
X = data['text']
y = data['clean_target']

In [ ]:
import warnings
import logging
from scipy.sparse import hstack
warnings.filterwarnings("ignore")


In [ ]:
multilabel_binarizer = MultiLabelBinarizer()
y_bin = multilabel_binarizer.fit_transform(y)

In [ ]:
y_bin

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
multilabel_binarizer.classes_

array(['accessories', 'action figures', 'animals', 'apparel',
       'appliances', 'artificial plants', 'assembly toys', 'baby',
       'baby care', 'backpacks', 'bags', 'bamboo plants', 'bangles',
       'bar accessories', 'barware', 'basket arrangements', 'bath items',
       'beauty', 'bed covers', 'bedsheets', 'belts', 'beverages',
       'birds paintings', 'biscuits', 'blooming plants', 'board games',
       'bonsai', 'bouquets', 'bouquets bunches', 'bows', 'bracelets',
       'branded chocolates', 'bunches', 'cakes', 'calendars', 'candies',
       'candles', 'carriers', 'cheese cakes', 'chess', 'chocolates',
       'clocks', 'clubs', 'clutches', 'cookies', 'covers', 'covers ',
       'creative play', 'cufflinks', 'cupcakes', 'cups', 'cushions',
       'decor', 'deos', 'designer cakes', 'desk accessories', 'diaries',
       'dining', 'dolls', 'dress materials', 'dresses', 'dry cakes',
       'dry fruits', 'dryfruits', 'earrings', 'educational games',
       'electronic toys', 'ele

In [ ]:
pd.DataFrame(y_bin, columns=multilabel_binarizer.classes_) 

,accessories,action figures,animals,apparel,appliances,artificial plants,assembly toys,baby,baby care,backpacks,...,trays,tshirts,vases,vehicle toys,wall decor,wallets,watches,wedding chudas,women western wear,wrist bands
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31958,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31959,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31960,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31961,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y_bin, test_size = 0.2, random_state = 0)

In [ ]:
vectorizer_char_tfidf = TfidfVectorizer(analyzer = 'char_wb',
                                       stop_words='english',
                                       ngram_range=(1,3),
                                       max_features=5000)

In [ ]:
X_tfidf_train_char = vectorizer_char_tfidf.fit(X_train)
X_tfidf_train_char = vectorizer_char_tfidf.transform(X_train)
X_tfidf_test_char = vectorizer_char_tfidf.transform(X_test)


In [ ]:
X_tfidf_test_char.shape

(6393, 5000)

In [ ]:
X_tfidf_train_char.shape

(25570, 5000)

In [ ]:
vectorizer_word_tfidf = TfidfVectorizer(analyzer = 'word',
                                       stop_words='english',
                                       ngram_range=(1,3),
                                       max_features=5000)

In [ ]:
X_tfidf_train_word = vectorizer_word_tfidf.fit(X_train)
X_tfidf_train_word = vectorizer_word_tfidf.transform(X_train)
X_tfidf_test_word = vectorizer_word_tfidf.transform(X_test)


In [ ]:
X_tfidf_train_word.shape, X_tfidf_test_word.shape

((25570, 5000), (6393, 5000))

In [ ]:
vectorizer_ch_tfidf = TfidfVectorizer(analyzer = 'char',
                                       stop_words='english',
                                       ngram_range=(1,3),
                                       max_features=5000)

In [ ]:
X_tfidf_train_ch = vectorizer_ch_tfidf.fit(X_train)
X_tfidf_train_ch = vectorizer_ch_tfidf.transform(X_train)
X_tfidf_test_ch = vectorizer_ch_tfidf.transform(X_test)


In [ ]:
X_tfidf_train_ch.shape, X_tfidf_test_ch.shape

((25570, 5000), (6393, 5000))

In [ ]:
def avg_jacard(y_true,y_pred):
    jacard = np.minimum(y_true,y_pred).sum(axis=1) / np.maximum(y_true,y_pred).sum(axis=1)    
    return jacard.mean()*100
    
def print_score(y_pred, clf):
    print("Clf: ", clf.__class__.__name__)
    print("Jacard score: {}".format(avg_jacard(y_test, y_pred)))
    print("Hamming loss: {}".format(hamming_loss(y_pred, y_test)*100))
    print("---")

In [ ]:
lr = LogisticRegression(solver = 'lbfgs')
svc = LinearSVC()
sgd = SGDClassifier()
rfc = RandomForestClassifier()

## Analyzer -> Char_wb with 5000 max_featur

In [ ]:
for classfier in [lr, svc,  sgd, rfc]:
  clf = OneVsRestClassifier(classfier)
  clf.fit(X_tfidf_train_char, y_train)
  y_pred = clf.predict(X_tfidf_test_char)
  print_score(y_pred, classfier)

Clf:  LogisticRegression
Jacard score: 75.06695070749505
Hamming loss: 0.3637514411016679
---
Clf:  LinearSVC
Jacard score: 87.14136346820534
Hamming loss: 0.2096477049550724
---
Clf:  SGDClassifier
Jacard score: 79.38823160432733
Hamming loss: 0.308207472293191
---
Clf:  RandomForestClassifier
Jacard score: 80.0035330173763
Hamming loss: 0.289596259798043
---


## Analyzer -> word with 5000 max_featur

In [ ]:
for classfier in [lr, svc,  sgd, rfc]:
  clf = OneVsRestClassifier(classfier)
  clf.fit(X_tfidf_train_word, y_train)
  y_pred = clf.predict(X_tfidf_test_word)
  print_score(y_pred, classfier)

Clf:  LogisticRegression
Jacard score: 75.54225468467608
Hamming loss: 0.34738516085301635
---
Clf:  LinearSVC
Jacard score: 85.71669128098034
Hamming loss: 0.22434839031116208
---
Clf:  SGDClassifier
Jacard score: 82.04678457611352
Hamming loss: 0.2657710111174838
---
Clf:  RandomForestClassifier
Jacard score: 83.64599154603847
Hamming loss: 0.25280833782319784
---


## Analyzer -> Char with 5000 max_featur

In [ ]:
for classfier in [lr, svc,  sgd, rfc]:
  clf = OneVsRestClassifier(classfier)
  clf.fit(X_tfidf_train_ch, y_train)
  y_pred = clf.predict(X_tfidf_test_ch)
  print_score(y_pred, classfier)

Clf:  LogisticRegression
Jacard score: 74.50785705595652
Hamming loss: 0.36939998030252996
---
Clf:  LinearSVC
Jacard score: 87.08210346408377
Hamming loss: 0.20928561910886329
---
Clf:  SGDClassifier
Jacard score: 79.50345841828714
Hamming loss: 0.30407969364640725
---
Clf:  RandomForestClassifier
Jacard score: 79.04023475351491
Hamming loss: 0.2997346634918979
---


I've compared with three analyzer in order to increase the accuracy. Hence, The best accuracy has found 87.1 with "char_wb"